## Tests

Comprehensive tests for logging decorators using mock handlers and context managers.

In [ ]:
#| hide
from numpy import all
from fh_saas.utils_log import TenantContext, get_context

### Test 1: TenantContext Context Manager

In [ ]:
#| hide
from fh_saas.utils_log import TenantContext, get_context

print("\n1️⃣ Testing TenantContext context manager...")

# Test 1a: Context is empty initially
ctx = get_context()
assert ctx['tenant_id'] is None
assert ctx['user_id'] is None
assert ctx['request_id'] is None
print("   ✅ Initial context is empty")

# Test 1b: Context is set within block
with TenantContext(tenant_id='tenant_123', user_id='user_456', request_id='req_789'):
    ctx = get_context()
    assert ctx['tenant_id'] == 'tenant_123'
    assert ctx['user_id'] == 'user_456'
    assert ctx['request_id'] == 'req_789'
    print("   ✅ Context set correctly within block")

# Test 1c: Context is cleared after block
ctx = get_context()
assert ctx['tenant_id'] is None
assert ctx['user_id'] is None
print("   ✅ Context cleared after block")


1️⃣ Testing TenantContext context manager...
   ✅ Initial context is empty
   ✅ Context set correctly within block
   ✅ Context cleared after block


### Test 2: DatabaseHandler

In [ ]:
#| hide
from fh_saas.utils_log import DatabaseHandler

print("\n2️⃣ Testing DatabaseHandler...")

# Test 2a: Handler collects logs
handler = DatabaseHandler()
handler.write_log('INFO', 'Test message', operation='test_op', status='success')

logs = handler.get_logs()
assert len(logs) == 1
assert logs[0]['level'] == 'INFO'
assert logs[0]['message'] == 'Test message'
assert logs[0]['operation'] == 'test_op'
print("   ✅ Handler collects logs correctly")

# Test 2b: Context is included in logs
handler.clear_logs()
with TenantContext(tenant_id='tenant_123', user_id='user_456'):
    handler.write_log('INFO', 'With context', operation='ctx_test')

logs = handler.get_logs()
assert logs[0]['tenant_id'] == 'tenant_123'
assert logs[0]['user_id'] == 'user_456'
print("   ✅ Context included in logs")

# Test 2c: Filter by level
handler.clear_logs()
handler.write_log('INFO', 'Info message')
handler.write_log('ERROR', 'Error message')
handler.write_log('INFO', 'Another info')

info_logs = handler.get_logs('INFO')
error_logs = handler.get_logs('ERROR')
assert len(info_logs) == 2
assert len(error_logs) == 1
print("   ✅ Filter by level works")


2️⃣ Testing DatabaseHandler...
   ✅ Handler collects logs correctly
   ✅ Context included in logs
   ✅ Filter by level works


### Test 3: @log_db_operation Decorator (Sync)

Each test creates its own handler for isolation.

In [ ]:
#| hide
from fh_saas.utils_log import log_db_operation

print("\n3️⃣ Testing @log_db_operation decorator (sync)...")

# Test 3a: Successful operation - isolated handler
def test_sync_db_operation_success():
    handler = DatabaseHandler()
    
    @log_db_operation('INSERT', handler=handler)
    def create_user(user_id: str, email: str):
        return {'id': user_id, 'email': email}
    
    with TenantContext(tenant_id='tenant_123'):
        result = create_user('user_789', 'test@example.com')
    
    # Verify function works
    assert result['id'] == 'user_789', "Function should return user_id"
    assert result['email'] == 'test@example.com', "Function should return email"
    
    # Verify logging
    logs = handler.get_logs()
    assert len(logs) == 1, "Should have exactly 1 log entry"
    log = logs[0]
    assert log['operation'] == 'create_user', "Operation name should match"
    assert log['status'] == 'success', "Status should be success"
    assert log['operation_type'] == 'INSERT', "Operation type should be INSERT"
    assert log['tenant_id'] == 'tenant_123', "Tenant context should be captured"
    assert 'duration_ms' in log, "Duration should be logged"
    
    print("   ✅ Success logged with context")

test_sync_db_operation_success()

# Test 3b: Failed operation - isolated handler
def test_sync_db_operation_error():
    handler = DatabaseHandler()
    
    @log_db_operation('DELETE', handler=handler)
    def delete_user(user_id: str):
        raise ValueError("User not found")
    
    exception_raised = False
    try:
        delete_user('nonexistent')
    except ValueError as e:
        exception_raised = True
        assert "User not found" in str(e), "Exception message should match"
    
    assert exception_raised, "Should have raised ValueError"
    
    # Verify error logged
    logs = handler.get_logs()
    assert len(logs) == 1, "Should have exactly 1 error log"
    log = logs[0]
    assert log['level'] == 'ERROR', "Log level should be ERROR"
    assert log['status'] == 'error', "Status should be error"
    assert log['operation_type'] == 'DELETE', "Operation type should be DELETE"
    assert log['error_type'] == 'ValueError', "Error type should be ValueError"
    assert 'User not found' in log['error_message'], "Error message should be captured"
    
    print("   ✅ Error logged with details")

test_sync_db_operation_error()


3️⃣ Testing @log_db_operation decorator (sync)...
   ✅ Success logged with context
   ✅ Error logged with details


### Test 4: @log_db_operation Decorator (Async)

Test async function support with isolated handler.

In [ ]:
#| hide
import asyncio
from fh_saas.utils_log import TenantContext, get_context

print("\n4️⃣ Testing @log_db_operation decorator (async)...")

async def test_async_db_operation():
    handler = DatabaseHandler()
    
    @log_db_operation('SELECT', handler=handler)
    async def fetch_user(user_id: str):
        await asyncio.sleep(0.001)  # Simulate async work
        return {'id': user_id, 'name': 'Test User'}
    
    # Run in isolated context
    with TenantContext(tenant_id='tenant_async'):
        result = await fetch_user('user_123')
    
    # Verify function works
    assert result['id'] == 'user_123', "Async function should return user_id"
    assert result['name'] == 'Test User', "Async function should return name"
    
    # Verify logging
    logs = handler.get_logs()
    assert len(logs) == 1, "Should have exactly 1 log entry"
    log = logs[0]
    assert log['operation'] == 'fetch_user', "Operation name should match"
    assert log['status'] == 'success', "Status should be success"
    assert log['operation_type'] == 'SELECT', "Operation type should be SELECT"
    assert log['tenant_id'] == 'tenant_async', "Tenant context should be captured"
    
    print("   ✅ Async operation logged correctly")

await test_async_db_operation()


4️⃣ Testing @log_db_operation decorator (async)...
   ✅ Async operation logged correctly


### Test 5: @log_api_call Decorator

Test API call logging with full context (tenant, user, request).

In [ ]:
#| hide
from fh_saas.utils_log import DatabaseHandler, log_api_call

print("\n5️⃣ Testing @log_api_call decorator...")

async def test_api_call_logging():
    handler = DatabaseHandler()
    
    @log_api_call('POST', handler=handler)
    async def create_user_endpoint(user_data: dict):
        await asyncio.sleep(0.001)
        return {'status': 'created', 'user': user_data}
    
    # Run with full context
    with TenantContext(tenant_id='tenant_api', user_id='admin_123', request_id='req_456'):
        result = await create_user_endpoint({'name': 'John'})
    
    # Verify function works
    assert result['status'] == 'created', "API should return status"
    assert result['user']['name'] == 'John', "API should return user data"
    
    # Verify logging
    logs = handler.get_logs()
    assert len(logs) == 1, "Should have exactly 1 log entry"
    log = logs[0]
    assert log['operation'] == 'create_user_endpoint', "Operation name should match"
    assert log['status'] == 'success', "Status should be success"
    assert log['http_method'] == 'POST', "HTTP method should be POST"
    assert log['tenant_id'] == 'tenant_api', "Tenant context captured"
    assert log['user_id'] == 'admin_123', "User context captured"
    assert log['request_id'] == 'req_456', "Request context captured"
    
    print("   ✅ API call logged with full context")

await test_api_call_logging()


5️⃣ Testing @log_api_call decorator...
   ✅ API call logged with full context


### Test 6: @log_background_task Decorator

Test background task logging with success and error cases.

In [ ]:
#| hide
from fh_saas.utils_log import log_background_task

print("\n6️⃣ Testing @log_background_task decorator...")

# Test 6a: Successful background task
async def test_background_task_success():
    handler = DatabaseHandler()
    
    @log_background_task('data_import', handler=handler)
    async def import_csv_data(file_path: str):
        await asyncio.sleep(0.001)
        return {'rows_imported': 1000, 'file': file_path}
    
    with TenantContext(tenant_id='tenant_bg'):
        result = await import_csv_data('/data/users.csv')
    
    # Verify function works
    assert result['rows_imported'] == 1000, "Should return rows count"
    assert result['file'] == '/data/users.csv', "Should return file path"
    
    # Verify logging
    logs = handler.get_logs()
    assert len(logs) == 1, "Should have exactly 1 log entry"
    log = logs[0]
    assert log['operation'] == 'data_import', "Operation name should match"
    assert log['status'] == 'success', "Status should be success"
    assert log['task_type'] == 'background', "Task type should be background"
    assert log['tenant_id'] == 'tenant_bg', "Tenant context captured"
    
    print("   ✅ Background task logged correctly")

await test_background_task_success()

# Test 6b: Failed background task
async def test_background_task_error():
    handler = DatabaseHandler()
    
    @log_background_task('email_batch', handler=handler)
    async def send_batch_emails():
        raise ConnectionError("SMTP server unavailable")
    
    exception_raised = False
    try:
        await send_batch_emails()
    except ConnectionError:
        exception_raised = True
    
    assert exception_raised, "Should have raised ConnectionError"
    
    # Verify error logged
    logs = handler.get_logs()
    assert len(logs) == 1, "Should have exactly 1 error log"
    log = logs[0]
    assert log['level'] == 'ERROR', "Log level should be ERROR"
    assert log['error_type'] == 'ConnectionError', "Error type captured"
    assert 'SMTP server' in log['error_message'], "Error message captured"
    
    print("   ✅ Background task error logged")

await test_background_task_error()


6️⃣ Testing @log_background_task decorator...
   ✅ Background task logged correctly
   ✅ Background task error logged


### Test 7: Decorator Stacking

Test that multiple decorators can be stacked on same function.

In [ ]:
#| hide
print("\n7️⃣ Testing decorator stacking...")

def test_multiple_decorators():
    """Test that decorators can be stacked without interference"""
    db_handler = DatabaseHandler()
    api_handler = DatabaseHandler()
    
    @log_api_call('POST', handler=api_handler)
    @log_db_operation('INSERT', handler=db_handler)
    def create_and_log(item_id: str):
        return {'id': item_id, 'created': True}
    
    with TenantContext(tenant_id='tenant_multi'):
        result = create_and_log('item_123')
    
    # Verify function still works
    assert result['id'] == 'item_123', "Function should return id"
    assert result['created'] is True, "Function should return created flag"
    
    # Verify both logs created independently
    db_logs = db_handler.get_logs()
    api_logs = api_handler.get_logs()
    
    assert len(db_logs) == 1, "DB handler should have 1 log"
    assert len(api_logs) == 1, "API handler should have 1 log"
    
    assert db_logs[0]['operation_type'] == 'INSERT', "DB log should have operation_type"
    assert api_logs[0]['http_method'] == 'POST', "API log should have http_method"
    
    print("   ✅ Decorators stack independently")

test_multiple_decorators()


7️⃣ Testing decorator stacking...
   ✅ Decorators stack independently


In [ ]:
#| hide
print("\n" + "="*60)
print("✅ ALL DECORATOR TESTS PASSED!")
print("="*60)
print("\nSummary:")
print("  • TenantContext: Context propagation ✅")
print("  • DatabaseHandler: Log collection ✅")
print("  • @log_db_operation: Sync + Async ✅")
print("  • @log_api_call: API endpoints ✅")
print("  • @log_background_task: Background jobs ✅")
print("  • Multiple decorators: Stacking ✅")


✅ ALL DECORATOR TESTS PASSED!

Summary:
  • TenantContext: Context propagation ✅
  • DatabaseHandler: Log collection ✅
  • @log_db_operation: Sync + Async ✅
  • @log_api_call: API endpoints ✅
  • @log_background_task: Background jobs ✅
  • Multiple decorators: Stacking ✅
